### Yield From - Throwing Exceptions

In [1]:
class CloseCoroutine(Exception):
    pass

In [7]:
def echo():
    try:
        while True:
            received = yield
            print(received)
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/or GeneratorExit thrown')
        return 'from a GeneratorExit'

In [3]:
e = echo()
next(e)

In [8]:
e.throw(CloseCoroutine)

CloseCoroutine: 

In [9]:
e = echo()
next(e)
e.close()

closed method was called/or GeneratorExit thrown


In [10]:
e = echo()
next(e)
e.throw(GeneratorExit)

closed method was called/or GeneratorExit thrown


StopIteration: from a GeneratorExit

In [11]:
def delegator():
    result = yield from echo()
    yield 'subgen closed and returned:', result
    print('delegator closing...')

In [12]:
d = delegator()
next(d)
d.send('hello')

hello


In [13]:
d.throw(CloseCoroutine)

('subgen closed and returned:', 'coro was closed')

In [14]:
class IgnoreMe(Exception):
    pass

In [16]:
def echo():
    try:
        while True:
            try:
                received = yield
                print(received)
            except IgnoreMe:
                yield "I'm ignoring you!"
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/or GeneratoRexit thrown!')
        return 'from a GeneratorExit'

In [17]:
d = delegator()
next(d)

In [18]:
d.send('python')

python


In [19]:
result = d.throw(IgnoreMe, 1000)

/tmp/ipykernel_783/390020266.py:1: DeprecationWarning: the (type, exc, tb) signature of throw() is deprecated, use the single-arg signature instead.
  result = d.throw(IgnoreMe, 1000)


In [20]:
result

"I'm ignoring you!"